# **Trabajo Práctico N°7**
**Ejercicio: BostonHousingPrice-Dimensionalidad**

**El objetivo de este ejercicio es utilizar distintos métodos para reducir la dimensionalidad a 2 dimensiones y ver cuales considera de mayor relevancia cada método.**

In [24]:
"""Importaciones"""
import pandas as pd
from sklearn.datasets import load_boston
from sklearn.preprocessing import StandardScaler
import numpy as np
from tabulate import tabulate
from sklearn.decomposition import PCA
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import train_test_split

**Cargo los valores. Quito las filas con valores Nan y estandarizo**

In [25]:
true_news = pd.read_csv("https://raw.githubusercontent.com/emmanueliarussi/DataScienceCapstone/master/3_MidtermProjects/ProjectBHP/data/housing_data.csv")
true_news.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,NaN,36.2


In [26]:
data = true_news.dropna()
data_std = StandardScaler().fit_transform(data)

#**Método 1.**

**Creo la matriz de covarianza.**

In [27]:
matriz_cov = np.cov(data_std.T)

**Hallo autovalores y autovectores de la matriz.** 

In [28]:
autovalores, autovectores = np.linalg.eig(matriz_cov)

**Calculo el porcentaje de varianza de las variables.**

In [29]:
varianza_tot = sum(autovalores)
var_ind = [(i /varianza_tot) for i in autovalores]
resultados =[[data.columns.values[0], var_ind[0]],
            [data.columns.values[1], var_ind[1]],
            [data.columns.values[2], var_ind[2]],
            [data.columns.values[3], var_ind[3]],
            [data.columns.values[4], var_ind[4]],
            [data.columns.values[5], var_ind[5]],
            [data.columns.values[6], var_ind[6]],
            [data.columns.values[7], var_ind[7]],
            [data.columns.values[8], var_ind[8]],
            [data.columns.values[9], var_ind[9]],
            [data.columns.values[10], var_ind[10]],
            [data.columns.values[11], var_ind[11]],
            [data.columns.values[12], var_ind[12]]]
print(tabulate(resultados, headers=["Nombres",  "Porcentaje"]))

Nombres      Porcentaje
---------  ------------
CRIM         0.472112
ZN           0.115383
INDUS        0.0962526
CHAS         0.0640082
NOX          0.0587932
RM           0.047487
AGE          0.0397575
DIS          0.0275923
RAD          0.00441544
TAX          0.0204173
PTRATIO      0.00950503
B            0.017062
LSTAT        0.0144916


**Viendo estos valores, se puede determinar que la primera y segunda variables (CRIM, ZN) son las más esenciales para el modelo.**

#**Método 2.**

**Ahora se puede hacer descomposición por valores singulares. En python es conocido como PCA.**

In [30]:
pca = PCA(n_components=2)
pca.fit(data_std)

PCA(n_components=2)

**Confirmo los valores de PCA.**

In [31]:
print(pca.explained_variance_ratio_)

[0.4721123  0.11538325]


**Se puede apreciar que los valores son iguales a los encontrados con el método 1.**

#**Método 3.**

**Eliminación de features recursivos.**

In [32]:
X = data.drop(["MEDV"], axis = 1)
y = data["MEDV"]
modelo = LinearRegression()
rfe = RFE(modelo,n_features_to_select=2)

In [33]:
fit_modelo=rfe.fit(X,y)
print(fit_modelo.get_support())

[False False False False  True  True False False False False False False
 False]


**Muestro los nombres de las variables más importantes para este método.**

In [34]:
print(data.columns.values[4]+", "+ data.columns.values[5]+".")

NOX, RM.


**Curiosamente, este método nos dice que los features más importantes son NOX y RM. A diferencia de los métodos usados anteriormente.**

#**Método 4.**

**Importancia de features.**

In [35]:
modelo = ExtraTreesRegressor()
modelo.fit(X,y)
print(modelo.feature_importances_)

[0.03474843 0.00656372 0.03743118 0.01786204 0.03516101 0.32430519
 0.01737424 0.02990072 0.018298   0.05198067 0.06522341 0.01779159
 0.3433598 ]


**Muestro los nombres de las variables más importantes para este método.**

In [36]:
print(data.columns.values[5] +", "+  data.columns.values[12]+".")

RM, LSTAT.


**Este método también da variables diferentes a las de los primeros 2 métodos y solo comparte la variable RM del método 3.**

#**Conclusión.**

**Se puede apreciar en los métodos usados anteriormente que no siempre se considera relevantes a las mismas features. Entonces es importante corroborar estos resultados antes de eliminar columnas.**

**Observación: Se podría realizar un análisis de como impacta cada método en los resultados finales para ver cual tuvo mejor rendimineto. Sin embargo, no es la idea con la que se encaró el ejercicio (solo se buscó reducir la dimensionalidad con diferentes métodos).** 